In [8]:
import psycopg2
import pandas as pd
import keys

In [9]:
letters = ['A','A']
price = ['0', '2']
period = ['1-21-2021', '2-21-2021']
trend_period = ['12-21-2020']

In [10]:
alphabets_in_lowercase=[]
for i in range(65,91):
    alphabets_in_lowercase.append(chr(i))
abc = alphabets_in_lowercase
print(abc)
ind_1 = abc.index(letters[0])
ind_2 = abc.index(letters[1]) + 1
abc = abc[ind_1:ind_2]
abc = str(abc).replace('[', '').replace(']', '')
print(abc)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
'A'


In [11]:
#abc = get_abc_arr(lets)
#query = f"select * from historical_stock_data where ticker_id in (select id from master_ticker_list where left(ticker, 1) in ('A', 'b'));"
query = f"select * from historical_stock_data as a where ticker_id in (select id from master_ticker_list where left(ticker, 1) in ({abc})) and '{period[0]}'::date < date::date and date::date < '{period[1] if trend_period != '' else trend_period}'::date limit 1000;"

In [12]:
conn = psycopg2.connect(database="postgres", user=keys.user, password=keys.password, host=keys.host, port="5432")
cur = conn.cursor()
#cur.execute(f"select * from master_ticker_list limit 5;")
cur.execute(query)
data = cur.fetchall()
#print(data)
# ticker_id |        open        |       close        |        high        |        low        |  volume  |        date 
df=pd.DataFrame(data,columns=['ticker_id','open', 'close', 'high','low','volume', 'date'])
print(df)
conn.commit()
conn.close()

     ticker_id                open               close                high  \
0    137291413   2.130000114440918  2.1700000762939453   2.180000066757202   
1    137291413  2.1600000858306885   2.069999933242798  2.1700000762939453   
2    137291413   2.259999990463257                 2.5   2.549999952316284   
3    137291413  2.4600000381469727  2.3499999046325684  2.5999999046325684   
4    137291413   2.319999933242798                2.25   2.450000047683716   
..         ...                 ...                 ...                 ...   
995   95073638  36.130001068115234   36.63999938964844   36.91999816894531   
996   95073638   36.66999816894531  36.400001525878906    37.1150016784668   
997   95073638   36.68000030517578  36.810001373291016                37.0   
998   95073638   36.97999954223633  36.869998931884766  37.439998626708984   
999   95073638    37.0099983215332   36.95000076293945  37.400001525878906   

                    low     volume                 date  
0    

In [14]:
#only include if close price within price bounds
ids = set(df['ticker_id'].to_list())
for _id in ids:
    ser = df[df['ticker_id'] == _id]
    maxed = max(ser['close'])
    mined = min(ser['close'])
    #print(mined,maxed)
    if mined > int(price[0]) and maxed > int(price[1]):
        #print(mined, maxed)
        df.drop(ser.index, inplace=True)
print(df)



120
     ticker_id                 open                close                high  \
20   299236405   1.1200000047683716   1.1699999570846558  1.2100000381469727   
21   299236405   1.1699999570846558   1.1399999856948853   1.190000057220459   
22   299236405   1.1200000047683716   1.1399999856948853  1.1799999475479126   
23   299236405   1.1200000047683716    1.100000023841858   1.149999976158142   
24   299236405   1.2200000286102295   1.2100000381469727  1.2799999713897705   
..         ...                  ...                  ...                 ...   
843  275096434  0.39899998903274536   0.5260000228881836  0.5339999794960022   
844  275096434   0.5249999761581421   0.2160000056028366  0.5249999761581421   
845  275096434   0.2160000056028366   0.2160000056028366  0.2160000056028366   
846  275096434   0.5239999890327454  0.22599999606609344  0.5239999890327454   
847  275096434   0.5170000195503235   0.5189999938011169  0.5189999938011169   

                     low     volume